In [1]:
import os
import csv
import numpy as np
import tensorflow as tf
from itertools import product
import keras
from keras_flops import get_flops

def get_total_number_of_neurons(model, include_output_layer):
    '''
    Args:
        model: Keras model
        include_output_layer: A boolean parameter. Whether or not to include output layer's neurons into the calculation

    Returns: number of neurons in the given model
    '''
    num_layers = len(model.layers)
    total_num_of_neurons = 0
    for layer_index in range(num_layers):
        layer = model.layers[layer_index]
        # since we multiply layer output dimensions, initial value is set to 1.
        num_neurons_in_layer = 1
        for i in range(1, len(layer.output.shape)):
            try:
                # when it is a valid layer to count neurons, an output dimension of the layer can be convertible to int.
                num_neurons_in_layer *= int(layer.output.shape[i])
            except Exception:
                # if the output dimension of layer cannot be convertible to int,
                # just pass that layer since it is not a valid layer to count neurons
                pass
        # if num_neurons_in_layer is not still 1, it means we have a valid layer to count neurons
        if not num_neurons_in_layer == 1:
            # when it is an output layer
            if layer_index == (num_layers - 1):
                if include_output_layer:
                    total_num_of_neurons += num_neurons_in_layer
            else:  # when it is not an output layer
                total_num_of_neurons += num_neurons_in_layer
    return total_num_of_neurons

model_num = 1
input_shape_range =range(32,512,2)

with open("para_count_GoogLeNet.csv","w") as csvfile:
         writer = csv.writer(csvfile)
         writer.writerows([["model_num", "input_horizontal", "input_vertical", "step","number_of_layers", "FLOPs_G", "Total_params_M", "Memory_M", "total_num_of_neurons_M"],])
path = "../GoogLeNet_h5" 
files= os.listdir(path) 

for shape_x in input_shape_range:
    shape_y = shape_x
    for step in range(1,11):
        file = "GoogLeNet_{input_x}_{input_y}_{step}.h5".format(input_x=shape_x,input_y=shape_y,step=step)
        if file in files: 
            model_name = str(file)
            if not os.path.isdir(file):
                model = keras.models.load_model("../GoogLeNet_h5/{}".format(model_name))
                num_neurons = get_total_number_of_neurons(model= model, include_output_layer= True)
                num_neurons_M = num_neurons/10**6
                # how to compute the memory allocated by the activations of a model
                batch = 1
                shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) 
                else 1 for s in l.output_shape])) 
                for l in model.layers]))
                memory = (shapes_count * 4 * batch)/10**6
                print(memory)
                layers_length = len(model.layers)
                Total_params = round(model.count_params()/10 ** 6,2)
                FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
                with open("para_count_GoogLeNet.csv","a+") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerows([[model_num, shape_x, shape_y, step, layers_length, FLOPs, Total_params, memory, num_neurons_M],])

                    model_num = model_num + 1

2022-05-16 04:43:20.088420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 04:43:20.088436: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-16 04:43:21.454341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 04:43:21.454357: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR

4.543972


2022-05-16 04:43:22.472970: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:43:22.473063: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:43:22.496606: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number

4.757012


...

2022-05-16 04:44:51.822673: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:44:51.822748: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:44:51.827658: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.005ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

33.624732


2022-05-16 04:44:53.348398: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:44:53.348469: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:44:53.354028: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi